<a href="https://colab.research.google.com/github/niikun/langchain_tutorial/blob/main/BuildChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Chatbot

## Overview  
LLMを使ったチャットボットの設計と実装の例を説明します。このチャットボットは会話をすることができ、過去のやり取りを記憶することができます。

このチャットボットは、言語モデルを使って会話をするだけです。他にもいくつか関連するコンセプトがあります：

- 会話型RAG：外部データソース上でチャットボット体験を可能にする。  
- エージェント： アクションを取ることができるチャットボットを構築する  

このチュートリアルでは、これら2つのより高度なトピックに役立つ基本的なことを説明しますが、必要に応じて直接スキップすることもできます。

## Concept
- ChatModels  
チャットボットのインターフェイスは、生のテキストではなくメッセージに基づいているため、テキストLLMではなくチャットモデルに最適です。
- PromptTemplates  
これは、デフォルトのメッセージ、ユーザー入力、チャット履歴、および (オプションで) 追加で取得したコンテキストを組み合わせたプロンプトを作成するプロセスを簡素化します。
- ChatHistory  
これは、チャットボットが過去のやりとりを "記憶 "し、フォローアップの質問に応答する際にそれらを考慮することを可能にします。
- LangSmith
LangSmithを使ったアプリケーションのデバッグとトレース.



## Setup
### Installation

In [2]:
!pip install openai
!pip install langchain
!pip install langchain-community
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

### Quickstart

In [3]:
!pip install -qU langchain-openai

In [4]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [6]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
model.invoke([HumanMessage(content="I got hungry.")])

AIMessage(content='Got it! Here are a few quick and easy snack ideas to satisfy your hunger:\n\n1. **Fruit**: An apple, banana, or a handful of berries can be both refreshing and satisfying.\n2. **Nuts**: A small handful of almonds, walnuts, or cashews can keep you full until your next meal.\n3. **Yogurt**: Pair it with some honey, granola, or fruit for a delicious treat.\n4. **Hummus and Veggies**: Carrot sticks, celery, and bell peppers with hummus make for a crunchy and nutritious snack.\n5. **Cheese and Crackers**: A few slices of cheese with whole-grain crackers can be quite satisfying.\n6. **Smoothie**: Blend some fruits, a bit of yogurt, and a splash of milk or juice for a quick and nutritious drink.\n\nDo any of these sound good to you? If you have specific ingredients on hand, I can suggest something more tailored!', response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 11, 'total_tokens': 212}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabde

In [8]:
AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-be38de4a-ccef-4a48-bf82-4292510a8cbf-0')

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-be38de4a-ccef-4a48-bf82-4292510a8cbf-0')

In [10]:
model.invoke(
    [
        HumanMessage(content="Hi,I am niikun.I am hungry."),
        AIMessage(content="Hello niikun!"),
        HumanMessage(content="What's my name?)")
    ]
)

AIMessage(content='You mentioned that your name is Niikun. How can I help you with your hunger? Are you looking for recipe ideas, nearby restaurants, or something else?', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 32, 'total_tokens': 64}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e2be63d-32c1-44a2-8322-a93bcb5e0ede-0')

### Message History  
Message Historyクラスを使ってモデルをラップし、ステートフルにすることができます。これはモデルの入力と出力を追跡し、いくつかのデータストアに保存します。将来のインタラクションでは、これらのメッセージを読み込んで、入力の一部としてチェーンに渡すことになります。これをどう使うか見てみましょう！

関連するクラスをインポートし、モデルをラップしてメッセージの履歴を追加するチェーンをセットアップします。  
ここで重要なのは、get_session_historyとして渡す関数です。この関数は session_id を受け取り、メッセージ履歴オブジェクトを返します。  
この session_id は個別の会話を区別するために使用され、新しいチェーンを呼び出すときに設定の一部として渡す必要があります。

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

ここで、毎回runnableに渡すコンフィグを作成する必要がある。  
このコンフィグには、直接入力の一部ではないが、有用な情報を含める。  
この場合、session_idを入れたい。これは次のようになります：

In [13]:
config = {"configurable": {"session_id": "abc2"}}

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, I am niikun. I am hungry.")], config=config
)
response

AIMessage(content="Hi Niikun! Let's find something to satisfy your hunger. Are you in the mood for a quick snack, a hearty meal, or maybe something sweet? Do you have any specific cravings or dietary restrictions? Let me know, and I'll help you with some suggestions!", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 172, 'total_tokens': 225}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa07c2a6-97be-44e1-9e58-cbc129efc383-0')

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)
response

AIMessage(content='Your name is Niikun. How can I assist you further?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 222, 'total_tokens': 235}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-0c973db3-d379-4046-bcc7-c55252cc0b2d-0')

In [22]:
store

{'abc2': InMemoryChatMessageHistory(messages=['Human: Hi, I am niikun. I am hungry.', AIMessage(content="Hi Niikun! If you're hungry, what kind of food are you in the mood for? I can give you some suggestions or even provide a recipe if you'd like to cook something yourself. Let me know what sounds good to you!", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 18, 'total_tokens': 65}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-f00c914d-0c19-456f-901c-d2b6a1f8d665-0'), AIMessage(content="Hi Niikun! If you're hungry, let's find something tasty for you. Are you in the mood for something specific, like a snack, a full meal, or maybe a dessert? Do you have any dietary preferences or restrictions? Let me know, and I can help you decide!", response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 86, 'total_tokens': 143}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43d

チャットボットは私たちのことを覚えています。別のsession_idを参照するように設定を変更すると、会話が新しく始まることがわかります。

In [23]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I can't determine your name based on the information provided. If you'd like to share your name or ask any other questions, feel free to do so!"

In [24]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You mentioned that your name is Niikun. How can I assist you today?'

### Prompt templates  
プロンプトテンプレートは、生のユーザー情報をLLMが扱える形式に変換するのに役立ちます。  
この場合、生のユーザー入力は単なるメッセージで、それをLLMに渡しています。  
もう少し複雑にしてみましょう。  
まず、いくつかのカスタム命令を含むシステム・メッセージを追加してみましょう（それでも入力としてメッセージを受け取ります）。次に、メッセージ以外の入力を追加します。  

まず、システム・メッセージを追加してみましょう。そのために、ChatPromptTemplateを作成します。MessagesPlaceholderを利用して全てのメッセージを渡します。

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [(
        "system","You are a helpful Osaka woman.Answer all questions in Japanese,Osaka language."
        ),
     MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [27]:
response = chain.invoke({"messages":[HumanMessage(content="おばちゃん、はらべったわ！")]})
response.content

'ほんまやなぁ、お腹すいたんやな。何食べたいん？たこ焼きとか、うどんとかどう？'

先ほどと同じように、これをメッセージ履歴オブジェクトにまとめることができます。

In [28]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [29]:
config = {"configurable": {"session_id": "abc5"}}

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="僕の名前はにいくん")],
    config=config,
)

response.content

'おお、にいくんやな！よろしくな、にいくん。何か手伝うことあったら、何でも言うてな。'

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="僕の名前はわかる？")],
    config=config,
)

response.content

'もちろんやで、にいくんやろ？覚えてるで！'

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="あなたの名前は？")],
    config=config
)
response.content

'ごめんな、わたしには特定の名前はあらへんねんけど、何か呼びたい名前あったら教えてくれたら、その名前で呼んでくれてええで。何かええ名前あるか？'

### Managing Conversation History

チャットボットを構築する際に理解すべき重要なコンセプトのひとつに、会話履歴の管理方法がある。

 管理しないままにしておくと、メッセージのリストは際限なく大きくなり、LLMのコンテキスト・ウィンドウをオーバーフローする可能性があります。
  
そのため、渡すメッセージのサイズを制限するステップを追加することが重要です。  

重要なのは、これをプロンプトテンプレートの前に実行し、メッセージ履歴から以前のメッセージを読み込んだ後に実行することである。  
これを行うには、プロンプトの前にメッセージキーを適切に変更する簡単なステップを追加し、その新しいチェーンを Message History クラスでラップします。まず、渡されたメッセージを変更する関数を定義しましょう。最新のk個のメッセージを選択するようにしましょう。そして、その関数を先頭に追加することで、新しいチェーンを作ることができます。

In [46]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

k = 10に設定しているので過去10個の会話は覚えている

In [47]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    # HumanMessage(content="whats 2 + 2"),
    # # AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [48]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'あんたの名前はボブやな！'

### Streaming
さて、チャットボットの機能ができました。  
しかし、チャットボットアプリケーションにとって本当に重要なUXの検討事項の1つは、ストリーミングです。  
LLMはレスポンスに時間がかかることがあるので、ユーザーエクスペリエンスを向上させるために、ほとんどのアプリケーションではトークンが生成されるたびにストリームバックしています。  
これにより、ユーザーは進捗状況を確認することができる。

実は、これを行うのはとても簡単だ！

すべてのチェーンは.streamメソッドを公開しており、メッセージ履歴を使用するものも同じです。このメソッドを使えば、ストリーミング・レスポンスを返すことができる。

In [49]:
config = {"configurable": {"session_id": "abc6"}}

for r in with_message_history.stream(
    [HumanMessage(content="Hi, I am niikun. I am hungry.")], config=config
):
    print(r.content,end="|")

|お|お|、|ニ|ーク|ン|や|ん|か|！|お|腹|空|いて|ん|ね|んな|ぁ|。|何|か|美|味|しい|も|ん|食|べ|に|行|こ|か|？|た|こ|焼|き|とか|、お|好|み|焼|き|とか|どう|や|？||